# Transfer Learning con EfficientNetV2-S - Versión 1 (V1)

Este notebook implementa la **Versión 1: Clasificador simple** usando Transfer Learning con EfficientNetV2-S según los requisitos del proyecto.

**Características de la Versión 1:**
- Modelo base: EfficientNetV2-S preentrenado en ImageNet
- **Clasificador simple: una sola capa Fully Connected**
- **Sin Batch Normalization adicional**
- **Sin Dropout**
- **Sin capas ocultas adicionales**
- Transfer Learning: Features congeladas, solo clasificador entrenado
- Early stopping basado en validation loss

**Objetivos:**
- Establecer baseline de rendimiento con arquitectura simple
- Evaluar efectividad del Transfer Learning
- Implementar pipeline completo de entrenamiento y evaluación

## 1. Configuración Inicial e Imports

**Importación de librerías necesarias:**
- PyTorch y torchvision para deep learning
- Sklearn para métricas de evaluación
- Matplotlib/Seaborn para visualización
- Funciones de utilidad personalizadas

**Configuración del entorno:**
- Detección automática de GPU/CPU
- Semillas aleatorias para reproducibilidad

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import time
import copy
import warnings

from utils.predict_images import predict_random_from_test, print_detailed_prediction, predict_single_image, visualize_prediction_result
from utils.train_model import train_model, plot_training_metrics
from utils.evaluate_model import evaluate_model
# Importar funciones de procesamiento de datos
from utils.data_proccess import create_transforms, load_datasets, create_dataloaders

warnings.filterwarnings('ignore')

# Configuracion del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Dispositivo utilizado: {device}')

# Configuracion para reproducibilidad
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## 2. Carga y Procesamiento de Datos

**Implementación de los requisitos de la sección 3.2:**
- **Transformaciones apropiadas**: Redimensionado a 224x224, normalización ImageNet
- **Data augmentation**: Aplicado en conjunto de entrenamiento
- **División de datos**: Train/Validation/Test usando funciones de utilidad

**Configuración de datos:**
- Batch size: 32
- Clases automáticamente detectadas del directorio
- DataLoaders optimizados para entrenamiento

In [ ]:
# Configuracion del proyecto
data_dir = "datos"
img_size = 224
batch_size = 32

# Crear transformaciones
print("Creando transformaciones...")
train_transform, val_test_transform = create_transforms(img_size=img_size)

# Cargar datasets
print("Cargando datasets...")
train_dataset, val_dataset, test_dataset = load_datasets(
    data_dir, train_transform, val_test_transform
)

# Mostrar informacion de las clases
print(f"\nClases detectadas: {train_dataset.classes}")
print(f"Numero de clases: {len(train_dataset.classes)}")
print(f"Muestras de entrenamiento: {len(train_dataset)}")
print(f"Muestras de validacion: {len(val_dataset)}")
print(f"Muestras de test: {len(test_dataset)}")

# Crear DataLoaders
print("\nCreando DataLoaders...")
train_loader, val_loader, test_loader = create_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=batch_size
)

print(f"Batches de entrenamiento: {len(train_loader)}")
print(f"Batches de validacion: {len(val_loader)}")
print(f"Batches de test: {len(test_loader)}")

# Guardar variables importantes
class_names = train_dataset.classes
num_classes = len(class_names)

## 3. Definición del Modelo V1 - Clasificador Simple

**Implementación de la Versión 1 según requisitos:**
- **Backbone**: EfficientNetV2-S preentrenado en ImageNet
- **Transfer Learning**: Features congeladas (solo clasificador entrenable)
- **Arquitectura simple**: Una sola capa Linear (in_features → num_classes)
- **Sin regularización**: No BatchNorm, No Dropout, No capas ocultas

**Características técnicas:**
- Aprovecha features preentrenadas de EfficientNetV2-S
- Mínimo número de parámetros entrenables
- Enfoque directo y eficiente para clasificación

In [ ]:
def build_model_v1(num_classes):
    """
    Construye el modelo EfficientNetV2-S V1 con transfer learning
    
    Args:
        num_classes (int): Numero de clases para el clasificador
    
    Returns:
        model: Modelo EfficientNetV2-S modificado para V1
    """
    # Cargar modelo preentrenado
    model = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
    
    # Congelar todas las capas del feature extractor
    for param in model.features.parameters():
        param.requires_grad = False
    
    # Obtener el numero de features de entrada del clasificador
    in_features = model.classifier[1].in_features
    
    # Reemplazar el clasificador con una sola capa Linear (V1)
    # Sin BatchNorm, sin Dropout, sin capas ocultas adicionales
    model.classifier = nn.Linear(in_features, num_classes)
    
    print(f"Modelo EfficientNetV2-S V1 creado exitosamente")
    print(f"Features de entrada del clasificador: {in_features}")
    print(f"Clases de salida: {num_classes}")
    print(f"Arquitectura V1: Una sola capa Linear({in_features} -> {num_classes})")
    
    return model

In [ ]:
# Construir modelo V1
print("Construyendo modelo EfficientNetV2-S V1...")
model = build_model_v1(num_classes)
model = model.to(device)

# Mostrar arquitectura del clasificador
print(f"\nArquitectura del clasificador:")
print(model.classifier)

# Verificar parametros entrenables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\nParametros totales: {total_params:,}")
print(f"Parametros entrenables: {trainable_params:,}")
print(f"Parametros congelados: {total_params - trainable_params:,}")
print(f"Porcentaje entrenables: {(trainable_params/total_params)*100:.2f}%")

# Configurar criterion y optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"\nConfiguración del entrenamiento:")
print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Early stopping: activado (patience=5)")

## 4. Entrenamiento con Early Stopping

**Cumplimiento de requisitos 3.2:**
- **Early stopping**: Patience=5 épocas para prevenir overfitting
- **Registro de pérdidas**: Train y validation loss automático
- **Optimizador**: Adam con learning rate 0.001
- **Función de pérdida**: CrossEntropyLoss

**Configuración de entrenamiento:**
- Épocas máximas: 10 (con early stopping)
- Monitoreo de validation loss para mejores pesos
- Tracking automático de accuracy y pérdidas

In [ ]:
# Entrenar el modelo
num_epochs = 10
patience = 5

print("="*60)
print("ENTRENAMIENTO DEL MODELO EFFICIENTNETV2-S V1")
print("="*60)

trained_model, train_losses, val_losses, train_accuracies, val_accuracies, best_epoch = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    patience=patience,
    device=device
)

print("\n" + "="*60)
print("ENTRENAMIENTO COMPLETADO")
print("="*60)

## 5. Visualización de Curvas de Entrenamiento

**Cumplimiento de requisitos 3.2:**
- **Gráficos de curvas de pérdida**: Train vs Validation loss
- **Curvas de accuracy**: Progreso durante entrenamiento
- **Identificación de mejor época**: Marcada en las gráficas
- **Análisis de convergencia**: Visual del comportamiento del modelo

**Métricas visualizadas:**
- Loss curves para detectar overfitting/underfitting
- Accuracy curves para evaluar mejora del modelo
- Best epoch marker para early stopping

In [ ]:
# Graficar curvas de perdida
print("Generando gráficas de métricas de entrenamiento...")
plot_training_metrics(train_losses, val_losses, train_accuracies, val_accuracies, best_epoch)

## 6. Evaluación Final en Conjunto de Prueba

**Cumplimiento de requisitos 3.3:**
- **Evaluación en conjunto de prueba**: Datos no vistos durante entrenamiento
- **Matriz de confusión**: Visualización con heatmap por clases
- **Métricas por clase**: Accuracy, Precision, Recall automáticos
- **Classification report**: Detallado con F1-score y support

**Objetivo:**
- Medir rendimiento real del modelo V1 en datos independientes
- Establecer baseline para comparación con Versión 2

In [ ]:
# Evaluacion completa del modelo
print("="*60)
print("EVALUACION FINAL EN CONJUNTO DE TEST")
print("="*60)

test_accuracy, y_true, y_pred = evaluate_model(
    model=trained_model,
    test_loader=test_loader,
    class_names=class_names,
    device=device
)

## 7. Guardado del Modelo Entrenado

**Persistencia del modelo:**
- Guardado de pesos del mejor modelo (best epoch)
- Metadatos incluidos: accuracies, losses, class names
- Estado del optimizador para posible continuación del entrenamiento

**Información guardada:**
- Model state dict con pesos entrenados
- Historial completo de entrenamiento
- Configuración de clases y métricas finales

In [ ]:
# Guardar el modelo entrenado
model_save_path = "efficientnetv2_s_v1.pth"

torch.save({
    'epoch': best_epoch,
    'model_state_dict': trained_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'test_accuracy': test_accuracy,
    'class_names': class_names,
    'num_classes': num_classes
}, model_save_path)

print(f"Modelo guardado exitosamente en: {model_save_path}")
print(f"Accuracy final en test: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

print("\n" + "="*60)
print("PROYECTO COMPLETADO - EFFICIENTNETV2-S V1")
print("="*60)
print(f"Mejor epoch: {best_epoch}")
print(f"Mejor validation loss: {min(val_losses):.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Clases: {class_names}")
print("="*60)

## 8. Sistema de Predicción Individual

**Funcionalidades implementadas:**
- Predicción de imágenes aleatorias del test set
- Visualización de predicciones con confianza
- Análisis detallado de resultados por imagen

**Herramientas disponibles:**
- Selección automática de muestras de prueba
- Visualización de imagen + predicción + ground truth
- Porcentajes de confianza por clase

In [ ]:
# OPCION 1: Prediccion de imagenes aleatorias del conjunto de test
print("\nPredicciones en imagenes aleatorias del conjunto de test")
print("-"*70)

test_dataset_path = f"{data_dir}/test"
num_test_samples = 3  # Puedes cambiar este numero

predict_random_from_test(
    model=trained_model,
    test_dataset_path=test_dataset_path,
    class_names=class_names,
    transform=val_test_transform,
    device=device,
    num_samples=num_test_samples
)

## 9. Demostración Interactiva del Modelo

**Funcionalidades de prueba:**
- **Predicciones aleatorias**: Automáticas del conjunto de test
- **Widget interactivo**: Selección manual de imágenes (si disponible)
- **Análisis de confianza**: Distribución de probabilidades por clase

**Propósito:**
- Validar funcionamiento del modelo entrenado
- Demostrar capacidades de clasificación
- Análisis cualitativo de predicciones

In [ ]:
# OPCION 2: Widget interactivo para seleccionar imágenes
print("\nOPCION 2: Selector interactivo de imágenes")
print("-"*70)

try:
    from utils.select_widget_imagen import setup_widget_prediction, display_prediction_widget
    
    # Configurar el widget con las variables del modelo
    setup_widget_prediction(
        model=trained_model,
        transform=val_test_transform,
        classes=class_names,
        device_type=device
    )
    
    # Mostrar el widget
    display_prediction_widget()
    
except ImportError:
    print("Error al importar el widget.")
    print("Verifica que ipywidgets esté instalado:")
    print("pip install ipywidgets")
except Exception as e:
    print(f"Error: {e}")
    print("Usando método alternativo...")